In [2]:
import collections
import math
import os
import random
import sys
import tarfile
import time
import json
from collections import namedtuple
import torch
from torch import nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import torchtext
import torchtext.vocab as Vocab
import numpy as np
import pandas as pd
from torch.utils.data import DataLoader, Dataset, WeightedRandomSampler
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
train = pd.read_csv('E:/LSTM_5_1/train_2018_1_model_2.csv')

dt = train['dt'].values

class MyData(Dataset):
    def __init__(self, traindata, numstep, length):
        self.serial_number = traindata['serial_number'].value_counts()
        self.value = traindata.iloc[:, length:-1].values
        max = np.max(self.value)
        min = np.min(self.value)
        scalar = max - min 
        self.datas = list(map(lambda x: x / scalar, self.value))
        self.input = []
        self.label = []
        for diskname in self.serial_number.index.sort_values():
            traindata_name = traindata[traindata.serial_number == diskname]
            self.datalabel = traindata_name.loc[:, 'label'].values.tolist()
            j = 0
            while(j + numstep <= len(traindata_name)):
                self.input.append(torch.Tensor(self.datas[j:j + numstep]))
                self.label.append(torch.Tensor(self.datalabel[j:j + numstep]))
                j = j + 7  
    def __len__(self):
        return len(self.input)
    def __getitem__(self, idx):
        #data = self.datas[idx].reshape(1, -1)
        return self.input[idx], self.label[idx]
train_data = pd.read_csv('E:/LSTM_5_1/train_2018_1_model_2.csv')
train_set = MyData(train_data, 7, 6)
train_iter = DataLoader(MyData(train_data, 7, 6), batch_size = 10)


In [151]:
train_data['label'].sum()

2049

In [156]:
num = 0
for X, y in train_set:
    num = num + y.sum()
num

tensor(1743.)

In [154]:
datalabel = train_data.loc[:, 'label'].values.tolist()
num = 0
for i in datalabel:
    num = num + i
num

2049

In [87]:
serial_number = train_data['serial_number'].value_counts()
serial_number_name = serial_number.index.sort_values()
serial_number_name.tolist().sort()
serial_number_name

Index(['disk_10421', 'disk_10590', 'disk_10707', 'disk_10893', 'disk_10969',
       'disk_11708', 'disk_117473', 'disk_117550', 'disk_117635',
       'disk_117717',
       ...
       'disk_9086', 'disk_9124', 'disk_9315', 'disk_9440', 'disk_9518',
       'disk_9761', 'disk_9801', 'disk_9807', 'disk_9810', 'disk_9996'],
      dtype='object', length=361)

In [97]:
serial_number = train_data['serial_number'].value_counts()
for i in serial_number.index.sort_values():
    print(train_data[train_data.serial_number == i])
    print(len(train_data[train_data.serial_number == i]))
    break
    

diff_day          dt  fault_time  label serial_number  \
0       96.0  2018-01-17  2018-04-23      0    disk_10421   
1       95.0  2018-01-18  2018-04-23      0    disk_10421   
2       94.0  2018-01-19  2018-04-23      0    disk_10421   
3       93.0  2018-01-20  2018-04-23      0    disk_10421   
4       92.0  2018-01-21  2018-04-23      0    disk_10421   
5       91.0  2018-01-22  2018-04-23      0    disk_10421   
6       90.0  2018-01-23  2018-04-23      0    disk_10421   
7       89.0  2018-01-24  2018-04-23      0    disk_10421   
8       88.0  2018-01-25  2018-04-23      0    disk_10421   
9       87.0  2018-01-26  2018-04-23      0    disk_10421   
10      86.0  2018-01-27  2018-04-23      0    disk_10421   
11      85.0  2018-01-28  2018-04-23      0    disk_10421   
12      84.0  2018-01-29  2018-04-23      0    disk_10421   
13      83.0  2018-01-30  2018-04-23      0    disk_10421   
14      82.0  2018-01-31  2018-04-23      0    disk_10421   

    smart_184_normalized  s

In [126]:
lenOfserial_number = []
for days in serial_number.index.sort_values():
    lenOfserial_number.append(len(train_data[train_data.serial_number == days]))


In [140]:
input = []
label = []
numstep = 7
for diskname in serial_number.index:
    j = 0
    train_data_name = train_data[train_data.serial_number == diskname]
    datalabel = train_data_name.loc[:, 'label'].values.tolist()
    while(j + numstep <= len(train_data[train_data.serial_number == diskname])):
        label.append(torch.Tensor(datalabel[j:j + numstep]))
        j = j + 7  
 

tensor(1743.)

In [162]:
a = []
for X, y in train_set:
    for i in y:
        a.append(i)
len(a)

8701

In [163]:
len(train_data)

9798

In [3]:
numDataPoints = 1000
data_dim = 5
bs = 100

# Create dummy data with class imbalance 9 to 1
data = torch.FloatTensor(numDataPoints, data_dim)
target = np.hstack((np.zeros(int(numDataPoints * 0.9), dtype=np.int32),
                    np.ones(int(numDataPoints * 0.1), dtype=np.int32)))

print('target train 0/1: {}/{}'.format(
    len(np.where(target == 0)[0]), len(np.where(target == 1)[0])))

class_sample_count = np.array(
    [len(np.where(target == t)[0]) for t in np.unique(target)])
weight = 1. / class_sample_count
samples_weight = np.array([weight[t] for t in target])

samples_weight = torch.from_numpy(samples_weight)
samples_weight = samples_weight.double()
sampler = WeightedRandomSampler(samples_weight, len(samples_weight))

target = torch.from_numpy(target).long()
train_dataset = torch.utils.data.TensorDataset(data, target)

train_loader = DataLoader(
    train_dataset, batch_size=bs, num_workers=1, sampler=sampler)

for i, (data, target) in enumerate(train_loader):
    print ("batch index {}, 0/1: {}/{}".format(
        i,
        len(np.where(target.numpy() == 0)[0]),
        len(np.where(target.numpy() == 1)[0])))
    

target train 0/1: 900/100
batch index 0, 0/1: 45/55
batch index 1, 0/1: 44/56
batch index 2, 0/1: 52/48
batch index 3, 0/1: 50/50
batch index 4, 0/1: 57/43
batch index 5, 0/1: 41/59
batch index 6, 0/1: 55/45
batch index 7, 0/1: 46/54
batch index 8, 0/1: 48/52
batch index 9, 0/1: 48/52


In [24]:
class_sample_count = np.array(
    [len(np.where(target == t)[0]) for t in np.unique(target)])
weight = 1. / class_sample_count
samples_weight = np.array([weight[t] for t in target])

samples_weight = torch.from_numpy(samples_weight)
samples_weight = samples_weight.double()
sampler = WeightedRandomSampler(samples_weight, len(samples_weight))



In [30]:
 [len(np.where(target == t)[0]) for t in np.unique(target)]

[56, 44]

In [42]:
for t in np.unique(target):
    print(t)
    print(np.where(target == t))
    print(type(np.where(target == t)))

0
(array([ 1,  2,  6,  7,  8,  9, 11, 12, 16, 17, 18, 21, 22, 23, 24, 25, 29,
       30, 34, 36, 37, 39, 40, 42, 44, 46, 48, 49, 50, 53, 59, 61, 63, 64,
       65, 66, 68, 69, 70, 73, 76, 77, 78, 80, 81, 82, 83, 84, 85, 87, 88,
       92, 93, 94, 95, 96], dtype=int64),)
<class 'tuple'>
1
(array([ 0,  3,  4,  5, 10, 13, 14, 15, 19, 20, 26, 27, 28, 31, 32, 33, 35,
       38, 41, 43, 45, 47, 51, 52, 54, 55, 56, 57, 58, 60, 62, 67, 71, 72,
       74, 75, 79, 86, 89, 90, 91, 97, 98, 99], dtype=int64),)
<class 'tuple'>


In [41]:
for t in np.unique(target):
    print(t)
    print(len(np.where(target == t)[0]))
    print(type(np.where(target == t)[0]))

0
56
<class 'numpy.ndarray'>
1
44
<class 'numpy.ndarray'>


In [43]:
class_sample_count

array([56, 44])

In [44]:
target

tensor([1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0,
        0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1,
        0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1,
        1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0,
        0, 1, 1, 1])

In [45]:
samples_weight

tensor([0.0227, 0.0179, 0.0179, 0.0227, 0.0227, 0.0227, 0.0179, 0.0179, 0.0179,
        0.0179, 0.0227, 0.0179, 0.0179, 0.0227, 0.0227, 0.0227, 0.0179, 0.0179,
        0.0179, 0.0227, 0.0227, 0.0179, 0.0179, 0.0179, 0.0179, 0.0179, 0.0227,
        0.0227, 0.0227, 0.0179, 0.0179, 0.0227, 0.0227, 0.0227, 0.0179, 0.0227,
        0.0179, 0.0179, 0.0227, 0.0179, 0.0179, 0.0227, 0.0179, 0.0227, 0.0179,
        0.0227, 0.0179, 0.0227, 0.0179, 0.0179, 0.0179, 0.0227, 0.0227, 0.0179,
        0.0227, 0.0227, 0.0227, 0.0227, 0.0227, 0.0179, 0.0227, 0.0179, 0.0227,
        0.0179, 0.0179, 0.0179, 0.0179, 0.0227, 0.0179, 0.0179, 0.0179, 0.0227,
        0.0227, 0.0179, 0.0227, 0.0227, 0.0179, 0.0179, 0.0179, 0.0227, 0.0179,
        0.0179, 0.0179, 0.0179, 0.0179, 0.0179, 0.0227, 0.0179, 0.0179, 0.0227,
        0.0227, 0.0227, 0.0179, 0.0179, 0.0179, 0.0179, 0.0179, 0.0227, 0.0227,
        0.0227], dtype=torch.float64)

In [46]:
weight

array([0.01785714, 0.02272727])

In [3]:
target

tensor([0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1,
        1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1,
        0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1,
        0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1,
        1, 1, 0, 1])

In [6]:
len(np.where(target == 0)[0])

44

In [9]:
for t in target:
    print(t)
    print(weight[t])

tensor(0)
0.0011111111111111111
tensor(0)
0.0011111111111111111
tensor(0)
0.0011111111111111111
tensor(0)
0.0011111111111111111
tensor(1)
0.01
tensor(1)
0.01
tensor(0)
0.0011111111111111111
tensor(0)
0.0011111111111111111
tensor(1)
0.01
tensor(0)
0.0011111111111111111
tensor(1)
0.01
tensor(1)
0.01
tensor(0)
0.0011111111111111111
tensor(0)
0.0011111111111111111
tensor(0)
0.0011111111111111111
tensor(0)
0.0011111111111111111
tensor(0)
0.0011111111111111111
tensor(0)
0.0011111111111111111
tensor(1)
0.01
tensor(1)
0.01
tensor(0)
0.0011111111111111111
tensor(0)
0.0011111111111111111
tensor(1)
0.01
tensor(1)
0.01
tensor(1)
0.01
tensor(1)
0.01
tensor(1)
0.01
tensor(0)
0.0011111111111111111
tensor(1)
0.01
tensor(0)
0.0011111111111111111
tensor(0)
0.0011111111111111111
tensor(1)
0.01
tensor(0)
0.0011111111111111111
tensor(0)
0.0011111111111111111
tensor(1)
0.01
tensor(0)
0.0011111111111111111
tensor(1)
0.01
tensor(1)
0.01
tensor(0)
0.0011111111111111111
tensor(1)
0.01
tensor(0)
0.00111111111111

In [12]:
weight

array([0.00111111, 0.01      ])

In [29]:
train_set = MyData(train_data, 7)

In [51]:
num = 0
for X, y in train_set:
    num = num + y.sum()
num      

tensor(0.)

In [38]:
train_data['label'].unique()

array([0, 1], dtype=int64)

In [46]:
train_data['label'].sum()

58

In [5]:
np.unique(target)

array([0, 1], dtype=int64)

In [6]:
samples_weight.shape

torch.Size([1000])